In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
mp_face = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True
)

pose = mp_pose.Pose()
hands = mp_hands.Hands(max_num_hands=2)


In [3]:
def analyze_eye_contact(face_landmarks, frame_width):
    nose_tip = face_landmarks.landmark[1]  # nose
    nose_x = nose_tip.x * frame_width
    center_x = frame_width / 2

    if abs(nose_x - center_x) < 40:
        return "Good"
    else:
        return "Looking Away"


In [4]:
prev_nose_x = None

def analyze_head_movement(face_landmarks, frame_width):
    global prev_nose_x
    nose_tip = face_landmarks.landmark[1]
    nose_x = nose_tip.x * frame_width

    if prev_nose_x is None:
        prev_nose_x = nose_x
        return "Stable"

    movement = abs(nose_x - prev_nose_x)
    prev_nose_x = nose_x

    if movement > 15:
        return "Excessive"
    else:
        return "Stable"


In [5]:
def analyze_posture(pose_landmarks):
    left_shoulder = pose_landmarks.landmark[11]
    right_shoulder = pose_landmarks.landmark[12]

    shoulder_diff = abs(left_shoulder.y - right_shoulder.y)

    if shoulder_diff < 0.05:
        return "Upright"
    else:
        return "Slouching"


In [6]:
prev_hand_y = None

def analyze_hand_movement(hand_landmarks):
    global prev_hand_y

    wrist = hand_landmarks.landmark[0]
    current_y = wrist.y

    if prev_hand_y is None:
        prev_hand_y = current_y
        return "Normal"

    movement = abs(current_y - prev_hand_y)
    prev_hand_y = current_y

    if movement > 0.03:
        return "Excessive"
    else:
        return "Normal"


In [7]:
def calculate_confidence(eye, head, posture, hand):
    score = 100

    if eye != "Good":
        score -= 20
    if head == "Excessive":
        score -= 20
    if posture != "Upright":
        score -= 20
    if hand == "Excessive":
        score -= 10

    return max(score, 0)


In [8]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_results = face_mesh.process(rgb)
    pose_results = pose.process(rgb)
    hand_results = hands.process(rgb)

    eye = "N/A"
    head = "N/A"
    posture = "N/A"
    hand_move = "N/A"

    if face_results.multi_face_landmarks:
        face_landmarks = face_results.multi_face_landmarks[0]
        eye = analyze_eye_contact(face_landmarks, w)
        head = analyze_head_movement(face_landmarks, w)

    if pose_results.pose_landmarks:
        posture = analyze_posture(pose_results.pose_landmarks)

    if hand_results.multi_hand_landmarks:
        hand_landmarks = hand_results.multi_hand_landmarks[0]
        hand_move = analyze_hand_movement(hand_landmarks)

    confidence = calculate_confidence(eye, head, posture, hand_move)

    cv2.putText(frame, f"Eye Contact: {eye}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.putText(frame, f"Head Movement: {head}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.putText(frame, f"Posture: {posture}", (20, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.putText(frame, f"Hand Movement: {hand_move}", (20, 160),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.putText(frame, f"Confidence Score: {confidence}", (20, 200),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)

    cv2.imshow("AI Interview Body Language Analyzer", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()
